In [40]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [41]:
import numpy as np # linear algebra
import os
import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader, Dataset # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F
from PIL import Image

Set Device

In [42]:
#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu or cpu

assert torch.backends.mps.is_available(), "GPU is not available, check the directions above (or disable this assertion to use CPU)"
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # use gpu or cpu

In [43]:
DEVICE

device(type='mps')

In [44]:
FAST_RUN = False
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
#DATA_PATH = "/content/drive/Shareddrives/cse 455/train_data"
DATA_PATH = "Cat_Dog_train_data/"
BATCH_SIZE=100
SAVE_TO = "Saved"


In [45]:
transform = transforms.Compose([
transforms.Resize((IMAGE_WIDTH, IMAGE_HEIGHT)),
transforms.ToTensor(),
transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))
])


In [46]:
data = datasets.ImageFolder(DATA_PATH, transform=transform)

AttributeError: module 'torch' has no attribute '_six'

In [17]:
data[1]

NameError: name 'data' is not defined

In [18]:
train_size = int(0.4 * len(data))
test_size = int(0.1 * len(data))
val_size = len(data) - train_size - test_size

NameError: name 'data' is not defined

In [ ]:
train_size, val_size, test_size

In [ ]:
train_set, val_set, test_set = torch.utils.data.random_split(data, [train_size, val_size, test_size])

In [ ]:
train_loader = DataLoader(
train_set,
batch_size=BATCH_SIZE,
shuffle=True
)

val_loader = DataLoader(
val_set,
batch_size=BATCH_SIZE,
shuffle=True
)

test_loader = DataLoader(
test_set,
batch_size=BATCH_SIZE,
shuffle=True
)

In [ ]:
class CatDogCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)

        # conected layers
        self.fc1 = nn.Linear(in_features= 2304, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)

        # flatten
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x 

In [ ]:
model = CatDogCNN().to(DEVICE)

In [ ]:
model

In [ ]:
optimizer = optim.Adam(model.parameters(), 0.0015)


In [ ]:
# online source

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def train():
    EPOCHS = 1

    loss = nn.CrossEntropyLoss()
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    # print_every = 100

    for e in range(EPOCHS):

      model.train()
      train_loss = 0.0
      train_acc = 0.0

      # itr = 0
      # start = time.time()

      for (x_batch, labels) in train_loader:
        x_batch, labels = x_batch.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        labels_pred = model(x_batch)
        batch_loss = loss(labels_pred, labels)
        train_loss = train_loss + batch_loss.item()

        labels_pred_max = torch.argmax(labels_pred, 1)
        batch_acc = torch.sum(labels_pred_max == labels)
        train_acc = train_acc + batch_acc.item()

        batch_loss.backward()
        optimizer.step()

        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(train_acc / (BATCH_SIZE * len(train_loader)))


        # itr = itr + 1
        # if iter % print_every == 0:
        #       print_loss_avg = train_loss / print_every
        #       print('%s (%d %d%%) %.4f' % (timeSince(start, iter / len(train_loader)),
        #                                     iter, iter / len(train_loader) * 100, print_loss_avg))
        # Validation loop; use .no_grad() context manager to save memory.
        model.eval()
        val_loss = 0.0
        val_acc = 0.0




        with torch.no_grad():
          for (v_batch, labels) in val_loader:
              v_batch, labels = v_batch.to(DEVICE), labels.to(DEVICE)
              labels_pred = model(v_batch)
              v_batch_loss = loss(labels_pred, labels)
              val_loss = val_loss + v_batch_loss.item()

              v_pred_max = torch.argmax(labels_pred, 1)
              batch_acc = torch.sum(v_pred_max == labels)
              val_acc = val_acc + batch_acc.item()
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(val_acc / (BATCH_SIZE * len(val_loader)))
      print("Epoch: {}, val loss: {:.4f}, val acc: {:.4f}, train loss: {:.4f}, train acc: {:.4f},\n".format(e, val_losses[e], val_accuracies[e], train_losses[e], train_accuracies[e]))


In [ ]:
CatDogCNN.train()
torch.save(CatDogCNN.state_dict(), SAVE_TO)

In [ ]:
def evaluate(model: nn.Module, loader: DataLoader) :
  loss = nn.CrossEntropyLoss()
  model.eval()
  test_loss = 0.0
  test_acc = 0.0
  with torch.no_grad():
    for (batch, labels) in loader:
      batch, labels = batch.to(DEVICE), labels.to(DEVICE)
      y_batch_pred = model(batch)
      batch_loss = loss(y_batch_pred, labels)
      test_loss = test_loss + batch_loss.item()

      pred_max = torch.argmax(y_batch_pred, 1)
      batch_acc = torch.sum(pred_max == labels)
      test_acc = test_acc + batch_acc.item()
    test_loss = test_loss / len(loader)
    test_acc = test_acc / (BATCH_SIZE * len(loader))
    return test_loss, test_acc

In [ ]:
test_loss, test_acc = evaluate(model, test_loader)
print(f"Test Accuracy: {test_acc}")